In [1]:
import os
import torch
import re
import unicodedata
import pdfplumber
#from pptx import Presentation
from transformers import AutoModel, AutoTokenizer
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings, get_response_synthesizer
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.node_parser import SentenceSplitter
from llama_index.vector_stores.chroma import ChromaVectorStore
from chromadb import PersistentClient
from llama_index.core.retrievers import BaseRetriever
from llama_index.core.query_engine import CustomQueryEngine
from llama_index.core.response_synthesizers import BaseSynthesizer
from llama_index.llms.ollama import Ollama
from llama_index.core.readers.base import BaseReader
from llama_index.core.schema import Document

c:\Users\kavet\Downloads\GNU\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ------------------------------
# CUSTOM PDF READER
# ------------------------------
class PDFTextOnlyReader(BaseReader):
    def load_data(self, file_path: str, extra_info=None) -> list[Document]:
        text_runs = []

        with pdfplumber.open(file_path) as pdf:
            for page in pdf.pages:
                text = page.extract_text()
                if text:
                    cleaned_text = preprocess_text(file_path, text)  # Apply preprocessing
                    text_runs.append(cleaned_text)

        full_text = "\n".join(text_runs)
        return [Document(text=full_text, metadata={"file_path": str(file_path)})]

In [3]:
# ------------------------------
# TEXT PREPROCESSING FUNCTION
# ------------------------------
def preprocess_text(file_path, text):
    code_extensions = {".py", ".java", ".cpp", ".js", ".c", ".cs", ".html", ".css", ".php", ".rb"}
    text_extensions = {".pdf", ".docx", ".pptx", ".txt"}

    ext = os.path.splitext(file_path)[-1].lower()

    if ext in text_extensions:
        text = clean_text(text)
    return text

def clean_text(text):
    text = unicodedata.normalize("NFKD", text)

    # Preserve email addresses
    email_pattern = r'([A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,})'
    emails = re.findall(email_pattern, text)
    for i, email in enumerate(emails):
        text = text.replace(email, f'EMAIL_PLACEHOLDER_{i}')

    # Preserve URLs
    url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    urls = re.findall(url_pattern, text)
    for i, url in enumerate(urls):
        text = text.replace(url, f'URL_PLACEHOLDER_{i}')

    # Remove table of contents artifacts (long sequences of dots)
    text = re.sub(r'\.{5,}', ' ', text)  # Remove sequences of 5 or more dots

    # Remove excessive spaces and unnecessary formatting
    text = re.sub(r'\s+', ' ', text).strip()  

    # Restore emails and URLs
    for i, email in enumerate(emails):
        text = text.replace(f'EMAIL_PLACEHOLDER_{i}', email)
    for i, url in enumerate(urls):
        text = text.replace(f'URL_PLACEHOLDER_{i}', url)

    return text

In [4]:
# ------------------------------
# MODEL & LLM SETTINGS
# ------------------------------
try:
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    Settings.embed_model = HuggingFaceEmbedding(model_name=model_name)
    print(f"✅ Model {model_name} loaded successfully.")
except Exception as e:
    print(f"❌ Error loading embedding model: {e}")
    exit()

Settings.llm = Ollama(model="llama3.1:latest", request_timeout=120.0)


✅ Model sentence-transformers/all-MiniLM-L6-v2 loaded successfully.


In [5]:
# ------------------------------
# LOAD DOCUMENTS & PREPROCESS
# ------------------------------
doc_path = "C:\\Users\\kavet\\Downloads\\GNU\\Data_Folder"

try:
    reader = SimpleDirectoryReader(
        input_dir=doc_path,
        required_exts=['.pptx', '.ipynb', '.docx', '.csv', '.jpeg', '.pdf', '.png', '.py'],
        file_extractor={".pdf": PDFTextOnlyReader()}, 
        recursive=True
    )
    docs = reader.load_data()

    if not docs:
        print("❌ No documents found! Check the path and file extensions.")
        exit()

    print(f"✅ Loaded {len(docs)} docs")

except Exception as e:
    print(f"❌ Error loading documents: {e}")
    exit()


✅ Loaded 3 docs


In [6]:
# ------------------------------
# PROCESS DOCUMENTS & CHUNKING
# ------------------------------
processed_docs = []
CHUNK_SIZE = 512
CHUNK_OVERLAP = 50

def chunk_text(text, chunk_size=512, overlap=50):
    chunks = []
    start = 0
    while start < len(text):
        end = min(start + chunk_size, len(text))
        chunk = text[start:end]
        chunks.append(chunk)
        start += chunk_size - overlap
    return chunks

for doc in docs:
    doc_text = preprocess_text(doc.metadata.get("file_path", ""), doc.get_content())
    text_chunks = chunk_text(doc_text)

    folder_name = os.path.basename(os.path.dirname(doc.metadata.get("file_path", "")))
    for chunk in text_chunks:
        metadata = {
            "file_name": os.path.basename(str(doc.metadata.get("file_path", ""))),
            "file_path": str(doc.metadata.get("file_path", "")),
            "folder_name": str(folder_name),
            "num_tokens": int(len(chunk.split())),
            "num_chars": int(len(chunk)),
        }
        processed_docs.append({
            "doc_id": doc.doc_id,
            "text": chunk,
            "metadata": metadata,
            "category": "<category>"
        })

In [7]:
# Print processed chunks
for i, doc in enumerate(processed_docs):
    print(f"\nChunk {i + 1}:")
    print(f"Document ID: {doc['doc_id']}")
    print(f"Metadata: {doc['metadata']}")
    print(f"Text: {doc['text']}\n")



Chunk 1:
Document ID: 6eaaf65d-a038-4dd4-8259-cdf7445b8236
Metadata: {'file_name': 'Disability_Rights_Laws_in_Public_Primary_and_Secondary_Education_final2018(2).pdf', 'file_path': 'C:\\Users\\kavet\\Downloads\\GNU\\Data_Folder\\Education\\Disability_Rights_Laws_in_Public_Primary_and_Secondary_Education_final2018(2).pdf', 'folder_name': 'Education', 'num_tokens': 69, 'num_chars': 512}
Text: National Network Call us toll-free 1-800-949-4232 V/TTY Information, Guidance and Training on the Find your regional center at Americans with Disabilities Act www.adata.org For the most current and accessible version, please visit https://adainfo.us/disability-rights. Disability Rights Laws in Public Primary and Secondary Education: How do they relate? Parents and caregivers of children with disabilities are often confused about which laws protect their children from discrimination and provide access to edu


Chunk 2:
Document ID: 6eaaf65d-a038-4dd4-8259-cdf7445b8236
Metadata: {'file_name': 'Disabi

In [8]:
# ------------------------------
# INITIALIZE CHROMADB VECTOR STORE
# ------------------------------
chroma_path = "./chroma_db"
try:
    chroma_client = PersistentClient(path=chroma_path)
    collection = chroma_client.get_or_create_collection("document_chunks")

    vector_store = ChromaVectorStore(chroma_client, collection_name="document_chunks")
    print("✅ ChromaDB initialized successfully.")
except Exception as e:
    print(f"❌ Error initializing ChromaDB: {e}")
    exit()

✅ ChromaDB initialized successfully.


In [9]:
from llama_index.core import Document

# List to hold the LlamaIndex nodes
nodes = []

# Loop through the processed_docs and create LlamaIndex Document nodes
for processed_doc in processed_docs:
    # Extract the text and metadata
    text = processed_doc["text"]
    metadata = processed_doc["metadata"]

    # Create a Document node
    doc_node = Document(
        text=text, 
        metadata=metadata
    )

    # Add the node to the nodes list
    nodes.append(doc_node)

# Now `nodes` contains the LlamaIndex nodes
print(f"✅ Created {len(nodes)} LlamaIndex nodes.")


✅ Created 66 LlamaIndex nodes.


In [10]:
# ------------------------------
# CREATE VECTOR STORE INDEX
# ------------------------------
try:
    index = VectorStoreIndex(nodes, vector_store=vector_store)
    print("✅ Vector store index created successfully.")
    persist_dir = "./persisted_index"
    os.makedirs(persist_dir, exist_ok=True)
    index.storage_context.persist(persist_dir=persist_dir)
    print(f"✅ Index persisted to {persist_dir}")
except Exception as e:
    print(f"❌ Error creating VectorStoreIndex: {e}")
    exit()

✅ Vector store index created successfully.
✅ Index persisted to ./persisted_index
